UK trade in services: all countries, non-seasonally adjusted

In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/businessindustryandtrade/' + \
                  'internationaltrade/datasets/uktradeinservicesallcountriesnonseasonallyadjusted')
    
tabs = {tab.name: tab for tab in scraper.distribution().as_databaker()}    

In [2]:
tab = tabs['TiS by country']

In [3]:
observations = tab.excel_ref('C7').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
Year = tab.excel_ref('C4').expand(RIGHT).is_not_whitespace()

In [5]:
Flow = tab.excel_ref('B').expand(DOWN).by_index([5,253])

In [6]:
geo1 = tab.excel_ref('A7').expand(DOWN).is_not_blank()

In [7]:
Dimensions = [
            HDim(Year,'Period',DIRECTLY,ABOVE),
            HDim(geo1,'ONS Partner Geography',DIRECTLY,LEFT),
            HDim(Flow, 'Flow',CLOSEST,ABOVE),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit','gbp-million')
            ]

In [8]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [9]:
new_table = c1.topandas()

In [10]:
new_table['Period'] = 'quarter/' + new_table['Period'].astype(str).str[0:4]+ '-' +   new_table['Period'].astype(str).str[-2:]              

In [11]:
new_table.head()

,OBS,Period,ONS Partner Geography,Flow,Measure Type,Unit
0,228.0,quarter/2016-Q1,AT,Exports,GBP Total,gbp-million
1,254.0,quarter/2016-Q2,AT,Exports,GBP Total,gbp-million
2,278.0,quarter/2016-Q3,AT,Exports,GBP Total,gbp-million
3,275.0,quarter/2016-Q4,AT,Exports,GBP Total,gbp-million
4,278.0,quarter/2017-Q1,AT,Exports,GBP Total,gbp-million


In [12]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [13]:
new_table['Flow'] = new_table['Flow'].map(lambda s: s.lower().strip())

In [14]:
new_table['Seasonal Adjustment'] =  'NSA'

In [16]:
new_table['Value'] = new_table['Value'].astype(int)

In [17]:
new_table['Pink Book Services'] = '0'

Duplicated observations of different countries in 

In [18]:
new_table = new_table.loc[new_table['ONS Partner Geography'].isin(['AD','AE','AF','AG','AI','AM','AO','AQ','AS','AW','AZ','BA','BB','BD','BF',	
                                                                'BH','BI','BJ','BM','BN','BO','BQ','BS','BT','BV','BW','BZ','CC','CD','CF',
                                                                'CG','CI','CK','CM','CR','CU','CV','CW','CX','D5','DJ','DM','DO','DZ','EC',
                                                                'ER','ET','FJ','FK','FM','FO','GA','GD','GE','GG','GH','GI','GL','GM','GN',
                                                                'GQ','GS','GT','GU','GW','GY','HM','HN','HT','IM','IO','IQ','JE','JM','JO',
                                                                'KE','KG','KH','KI','KM','KN','KP','KW','KY','KZ','LA',	'LB','LC','LK','LR',
                                                                'LS','LY','MD','MG','MH','MK','ML','MM','MN','MO','MP','MR','MS','MU','MV',
                                                                'MW','MZ','NA','NC','NE','NF','NG','NI','NP','NR','NU','OM','PA','PE','PF',
                                                                'PG','PN','PS','PW','PY','QA','RW','SB','SC','SD','SH','SL','SM','SN','SO',
                                                                'SR','SS','ST','SV','SX','SY','SZ','TC','TD','TF','TG','TJ','TK','TL','TM',
                                                                'TN','TO','TT','TV','TZ','UG','UM','UZ','VA','VC','VG','VI','VN','VU','WF',
                                                                'WS','XK','YE','ZM','ZW'])]

In [19]:
new_table = new_table[['ONS Partner Geography', 'Period','Flow','Pink Book Services','Seasonal Adjustment', 'Measure Type','Value','Unit' ]]